In [7]:
###importing necessary libraries 
import os 
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.image import resize


In [8]:
#Loading Model
model = tf.keras.models.load_model("./Training_model.h5")

In [9]:
classes = ["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]


In [10]:
###SINGLE AUDIO PREPROCESSING
###https://www.youtube.com/watch?v=knla9q-9Cn0&list=PLvz5lCwTgdXCd200WNDupTMo15DP9iryv&index=14
#LOAD and preprocess audio file
def load_and_preprocess_file(file_path,target_shape=(150,150)):
    data= []
    audio_data,sample_rate = librosa.load(file_path,sr=None)
    chunk_duration = 4
    overlap_duration = 2
    #convert duration to sample
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate
    #calculate the number of chunks
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1
    #iterate each chunk
    for i in range(num_chunks):
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
        chunk = audio_data[start:end]

        if len(chunk) < chunk_samples:
            chunk = np.pad(chunk, (0, chunk_samples - len(chunk)))

        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        with tf.device('/CPU:0'):  # Resize on CPU
            mel_spectrogram = tf.image.resize(
                np.expand_dims(mel_spectrogram, axis=-1), target_shape)
        data.append(mel_spectrogram)
    return np.array(data)

In [11]:
file_path = "./Test_music/Bensound_ Happy Rock - Royalty Free Music.mp3"
X_test = load_and_preprocess_file(file_path)

In [12]:
X_test.shape

(52, 150, 150, 1)

In [13]:

##MODEL PREDICTION
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred,axis=1)
    unique_elements,counts = np.unique(predicted_categories,return_counts=True)
    max_count = np.max(counts)
    max_elements = unique_elements[counts==max_count]
    return max_elements[0]

In [14]:
c_index= model_prediction(X_test)
c_index

2/2 [==============================] - 10s 1s/step


9

In [15]:
print("model_prediction:Music Prediction --->",classes[c_index])

model_prediction:Music Prediction ---> rock
